# Basic Examples of _findiff_

_findiff_ works in any dimension. But for the sake of demonstration, let's concentrate on the cases 1D and 3D. We are using uniform, i.e. equidistant, grids here. The non-uniform case will be shown in another notebook.

## Preliminaries

Our imports:

In [4]:
from jax import numpy as jnp

from fdx import Diff

## Simple 1D Cases

Suppose we want to differentiate two 1D-arrays `f` and `g`, which are filled with values from a function

$$
f(x) = \sin(x) \quad \text{and}\quad g(x) = \cos(x)
$$

and we want to take the 2nd derivative. This is easy done analytically:

$$
\frac{d^2f}{dx^2} = -\sin(x) \quad \text{and}\quad \frac{d^2g}{dx^2} = -\cos(x)
$$

Let's do this numerically with _findiff_. First we set up the grid and the arrays:

In [5]:
x = jnp.linspace(0, 10, 100)
dx = x[1] - x[0]
f = jnp.sin(x)
g = jnp.cos(x)

Then we construct the derivative object, which represents the differential operator $\frac{d^2}{dx^2}$:

In [6]:
d2_dx2 = Diff(0, dx, acc=2)

The first parameter is the axis along which to take the derivative. Since we want to apply it to the one and only axis of the 1D array, this is a 0. The second parameter is the grid spacing, the third parameter the derivative order you want, in our case 2. If you want a first derivative, you can skip the third argument as it defaults to 1.

Then we apply the operator to f and g, respectively:

In [7]:
result_f = d2_dx2(f)
result_g = d2_dx2(g)

That's it! The arrays `result_f`and `result_g` have the same shape as the arrays `f` and `g` and contain the values of the second derivatives.

#### Finite Difference Coefficients

By default the `FinDiff` class uses second order accuracy. For the second derivative, it uses the following finite difference coefficients:

In [8]:
from fdx import coefficients

coefficients(deriv=2, acc=2)

{'center': {'coefficients': Array([ 1., -2.,  1.], dtype=float32),
  'offsets': Array([-1,  0,  1], dtype=int32),
  'accuracy': 2},
 'forward': {'coefficients': Array([ 2.       , -4.9999995,  3.9999995, -0.9999999], dtype=float32),
  'offsets': Array([0, 1, 2, 3], dtype=int32),
  'accuracy': 2},
 'backward': {'coefficients': Array([-1.0000002,  4.000001 , -5.0000014,  2.000001 ], dtype=float32),
  'offsets': Array([-3, -2, -1,  0], dtype=int32),
  'accuracy': 2}}

But `FinDiff` can handle any accuracy order. For instance, have you ever wondered, what the 10th order accurate coefficients look like? Here they are:

In [9]:
coefficients(deriv=2, acc=10)

{'center': {'coefficients': Array([ 3.1744901e-04, -4.9601770e-03,  3.9681721e-02, -2.3809293e-01,
          1.6666629e+00, -2.9272256e+00,  1.6666741e+00, -2.3809946e-01,
          3.9683565e-02, -4.9604829e-03,  3.1747323e-04], dtype=float32),
  'offsets': Array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=int32),
  'accuracy': 1},
 'forward': {'coefficients': Array([  3.886053  , -12.239338  ,  14.595838  ,  -6.3827343 ,
          -4.203699  ,   8.571215  ,  -5.951184  ,   1.7408129 ,
           0.26981813,  -0.4003601 ,   0.12865865,  -0.0150808 ],      dtype=float32),
  'offsets': Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32),
  'accuracy': 1},
 'backward': {'coefficients': Array([  -0.5537981,    6.6648746,  -36.79572  ,  123.26714  ,
         -279.22177  ,  450.90286  , -532.983    ,  465.73615  ,
         -299.93546  ,  140.03792  ,  -44.696243 ,    7.5770097],      dtype=float32),
  'offsets': Array([-11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -

#### Accuracy order

If you want to use for example 10th order accuracy, just tell the `FinDiff` constructor to use it:

In [10]:
from fdx import FinDiff

d2_dx2 = FinDiff(0, dx, 2, acc=10)
result = d2_dx2(f)

## Simple 3D Cases

Now let's differentiate a 3D-array `f` representing the function

$$
f(x, y, z) = \sin(x) \cos(y) \sin(z) 
$$


In [11]:
x, y, z = [jnp.linspace(0, 10, 100)]*3
dx, dy, dz = x[1] - x[0], y[1] - y[0], z[1] - z[0]
X, Y, Z = jnp.meshgrid(x, y, z, indexing='ij')
f = jnp.sin(X) * jnp.cos(Y) * jnp.sin(Z)

The partial derivatives $\frac{\partial f}{\partial x}$ or $\frac{\partial f}{\partial z}$ are given by

In [12]:
d_dx = FinDiff(0, dx)
d_dz = FinDiff(2, dz)

The x-axis is the 0th axis, y, the first, z the 2nd, etc. The third mixed partial derivative $\frac{\partial^3 f}{\partial x^2 \partial y}$ is specified by two tuples as arguments, one for each partial derivative:

In [13]:
d3_dx2dy = FinDiff((0, dx, 2), (1, dy))
result = d3_dx2dy(f)

NotFoundLookupError: `lmul(<Diff with shape=(0, 0), dtype=float32>, <Diff with shape=(1, 1), dtype=float32>)` could not be resolved.

Closest candidates are the following:
    lmul(a: [1;31mint | float | complex | numpy.number | jax.Array[0m, b: [38;5;248mlinox._linear_operator.[0m[1;38;5;248mLinearOperator[0m) ->         
    [38;5;248mlinox._linear_operator.[0m[1;38;5;248mLinearOperator[0m                                                                          
        <function lmul at 0x10c8d63e0> @ ]8;id=689512;file:///Users/lenardrommel/repos/fdx/.venv/lib/python3.12/site-packages/linox/_arithmetic.py#46\[37m~/repos/fdx/.venv/lib/python3.12/site-packages/linox/[0m]8;;\]8;id=420143;file:///Users/lenardrommel/repos/fdx/.venv/lib/python3.12/site-packages/linox/_arithmetic.py#46\[1;4;37m_arithmetic.py[0m]8;;\]8;id=689512;file:///Users/lenardrommel/repos/fdx/.venv/lib/python3.12/site-packages/linox/_arithmetic.py#46\[37m:46[0m]8;;\    
    lmul(a: [31mjax

Of course, the accuracy order can be specified the same way as for 1D.

## General Linear Differential Operators

`FinDiff` objects can bei added and easily multiplied by numbers. For example, to express

$$
\frac{\partial^2}{\partial x^2} + 2\frac{\partial^2}{\partial x \partial y} + \frac{\partial^2}{\partial y^2} =
\left(\frac{\partial}{\partial x} + \frac{\partial}{\partial y}\right) \left(\frac{\partial}{\partial x} + \frac{\partial}{\partial y}\right)
$$

we can say

In [2]:
linear_op = FinDiff(0, dx, 2) + 2 * FinDiff((0, dx), (1, dy)) + FinDiff(1, dy, 2)

NameError: name 'FinDiff' is not defined

#### Variable Coefficients

If you want to multiply by variables instead of plain numbers, you have to encapsulate the variable in a `Coefficient` object. For example, 

$$
x \frac{\partial}{\partial x} + y^2 \frac{\partial}{\partial y}
$$

is

In [15]:
from fdx import Coefficient

linear_op = Coefficient(X) * FinDiff(0, dx) + Coefficient(Y**2) * FinDiff(1, dy)

ImportError: cannot import name 'Coefficient' from 'fdx' (/Users/lenardrommel/repos/fdx/fdx/__init__.py)

Applying those general operators works the same way as for the simple derivatives:

In [26]:
result = linear_op(f)